## Importing necessary libraries

In [152]:
import pandas as pd
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import pickle as pkl
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("averaged_perceptron_tagger_eng")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\rudra\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       d

True

## merging two dataframes 

In [114]:
df1 = pd.read_csv("spam.csv", encoding="latin1")
df2 = pd.read_csv("spam_ham_india.csv")
df1.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [115]:
# df1.drop("Unnamed: 2", axis=1)
df1 = df1[["v1", "v2"]]
df1 = df1[["v1", "v2"]].rename(columns={"v1": "Label", "v2": "Message"})
df1

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [116]:
df1_clean = df1[["Message", "Label"]]
df1_clean

,Message,Label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam
5568,Will Ì_ b going to esplanade fr home?,ham
5569,"Pity, * was in mood for that. So...any other s...",ham
5570,The guy did some bitching but I acted like i'd...,ham


In [117]:
df2.head()

,Msg,Label
0,CONGRATULATIONS! FREE 2GB data is yours! Claim...,spam
1,Hi! Thank you for being with Vi-India's FASTES...,spam
2,"As part of Cyber Swachhta Pakhwada, CERT-In Go...",spam
3,I will try to manage took tablets,ham
4,Study from Home with Vi!! Watch Kite Victers C...,spam


In [118]:
df2 = df2.rename(columns={"Msg": "Message"})
df2

,Message,Label
0,CONGRATULATIONS! FREE 2GB data is yours! Claim...,spam
1,Hi! Thank you for being with Vi-India's FASTES...,spam
2,"As part of Cyber Swachhta Pakhwada, CERT-In Go...",spam
3,I will try to manage took tablets,ham
4,Study from Home with Vi!! Watch Kite Victers C...,spam
...,...,...
2262,"Dear Customer, You have a missed call from +91...",spam
2263,"Dear User, Vistor Id - 7538XXX. Loan Applicati...",spam
2264,"Congrats, Y0UR Received Rs.592000 L0AN is Appr...",spam
2265,Hi! 50% off on proc. fees will be gone soon! A...,spam


In [119]:
df = pd.concat([df1_clean,df2], ignore_index=True)
df

,Message,Label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
7834,"Dear Customer, You have a missed call from +91...",spam
7835,"Dear User, Vistor Id - 7538XXX. Loan Applicati...",spam
7836,"Congrats, Y0UR Received Rs.592000 L0AN is Appr...",spam
7837,Hi! 50% off on proc. fees will be gone soon! A...,spam


In [120]:
df.isnull().sum()

Message    1
Label      0
dtype: int64

In [121]:
df = df.dropna()

In [122]:
df = df.drop_duplicates(subset=["Message"])

In [123]:
df.shape

(7230, 2)

In [124]:
df["Label"].value_counts()

Label
ham     5841
spam    1389
Name: count, dtype: int64

In [125]:
df.isnull().sum()

Message    0
Label      0
dtype: int64

In [126]:
df.dropna()

,Message,Label
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham
...,...,...
7834,"Dear Customer, You have a missed call from +91...",spam
7835,"Dear User, Vistor Id - 7538XXX. Loan Applicati...",spam
7836,"Congrats, Y0UR Received Rs.592000 L0AN is Appr...",spam
7837,Hi! 50% off on proc. fees will be gone soon! A...,spam


## Text Preprocessing

In [127]:
## converting text into lower
def convert_lower(text):
    return text.lower()

In [128]:
# removing punctuatios and special characters 
def remove_punctuations_specchars(text):
    result = re.sub(r'[^a-zA-Z\s]', '',text)
    return result



In [129]:
## removing stop words 

def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return " ".join(filtered_tokens)



In [130]:
## lemmatization of the words 

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'
    elif tag.startswith("V"):
        return 'v'
    elif tag.startswith("N"):
        return 'n'
    elif tag.startswith("R"):
        return 'r'
    else:
        return 'n'

def lemmatizer(text):
    lemmatize = WordNetLemmatizer()
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lematized_sentence = []
    for word,tag in tagged_tokens:
        if word.lower() == "are" or word.lower() in ["is", "am"]:
            lematized_sentence.append(word)
        else:
            lematized_sentence.append(lemmatize.lemmatize(word, get_wordnet_pos(tag)))
    return " ".join(lematized_sentence)



## Applying this function to out dataframe

In [131]:
df['Cleaned_messge'] = df["Message"].apply(convert_lower).apply(remove_punctuations_specchars).apply(remove_stop_words).apply(lemmatizer)

In [132]:
df.head()

,Message,Label,Cleaned_messge
0,"Go until jurong point, crazy.. Available only ...",ham,go jurong point crazy available bugis n great ...
1,Ok lar... Joking wif u oni...,ham,ok lar joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,free entry wkly comp win fa cup final tkts st ...
3,U dun say so early hor... U c already then say...,ham,u dun say early hor u c already say
4,"Nah I don't think he goes to usf, he lives aro...",ham,nah dont think go usf life around though


## vectorizing the data 

In [156]:
vector = TfidfVectorizer(max_features=15000)
X = vector.fit_transform(df["Cleaned_messge"])
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 61537 stored elements and shape (7230, 9818)>

In [157]:
with open("vectorizer.pkl", "wb") as file:
    pkl.dump(vector, file)

## Splitting Datasets 

In [134]:
## using train_test_split to split the dataset
le = LabelEncoder()
y = le.fit_transform(df["Label"])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [135]:
X_train.shape

(5784, 9818)

In [136]:
y_train.shape

(5784,)

## Creating a model


In [137]:
nb = MultinomialNB()


In [138]:
nb.fit(X_train, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [139]:
y_predict = nb.predict(X_test)
accuracy = accuracy_score(y_predict, y_test)
classification = classification_report(y_predict, y_test)
print(accuracy)
print(classification)

0.9605809128630706
              precision    recall  f1-score   support

           0       1.00      0.95      0.98      1192
           1       0.82      0.99      0.90       254

    accuracy                           0.96      1446
   macro avg       0.91      0.97      0.94      1446
weighted avg       0.97      0.96      0.96      1446



## checking model on real time data 

In [145]:
test_message = [
    # Indian Ham
    "Hey, let’s meet at CCD near MG Road at 5 pm.",
    "Don’t forget to bring your Aadhaar card for tomorrow’s appointment.",
    "Train is delayed by 30 minutes, I will reach CST at 10 pm.",
    "Can you recharge my Jio number with ₹199 pack?",

    # Indian Spam
    "Congratulations! You have won a free recharge worth ₹500, click here to claim.",
    "Get instant loan up to ₹5,00,000 with zero documents, apply now!",
    "Your SBI account is suspended, update KYC immediately: http://fake-link.com",
    "Exclusive deal on Flipkart! Flat 70% off on iPhones. Hurry, limited stock.",

    # Non-Indian Ham
    "Hey, are we still on for lunch today at Starbucks?",
    "Don’t forget to bring the documents for tomorrow’s meeting.",
    "I’ll be late to the office, stuck in traffic.",
    "Happy Birthday! Wishing you a great year ahead.",

    # Non-Indian Spam
    "Congratulations! You have won a free iPhone. Click here to claim now.",
    "Urgent: Your account has been suspended, update your details immediately.",
    "You’ve been selected for a $1000 Walmart gift card, claim today!",
    "Exclusive offer: Get 80% discount on Nike shoes. Visit http://fake-site.com"
]


In [146]:
def preprocess_text(text):
    text = convert_lower(text)
    text = remove_punctuations_specchars(text)
    text = remove_stop_words(text)
    text = lemmatizer(text)
    return text

cleaned_messge = [preprocess_text(text) for text in test_message]

In [147]:
cleaned_messge

['hey let meet ccd near mg road pm',
 'dont forget bring aadhaar card tomorrows appointment',
 'train delay minute reach cst pm',
 'recharge jio number pack',
 'congratulation free recharge worth click claim',
 'get instant loan zero document apply',
 'sbi account suspend update kyc immediately httpfakelinkcom',
 'exclusive deal flipkart flat iphones hurry limit stock',
 'hey still lunch today starbucks',
 'dont forget bring document tomorrows meeting',
 'ill late office stuck traffic',
 'happy birthday wish great year ahead',
 'congratulation free iphone click claim',
 'urgent account suspend update detail immediately',
 'youve select walmart gift card claim today',
 'exclusive offer get discount nike shoe visit httpfakesitecom']

In [148]:
real_message = vector.transform(cleaned_messge)
real_message

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 93 stored elements and shape (16, 9818)>

In [149]:
prediction = nb.predict(real_message)
pred = le.inverse_transform(prediction)
pred

array(['ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham',
       'ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'spam'], dtype=object)

## Creating a pickle file


In [153]:
with open("smsdetection.pkl", "wb") as file:
    pkl.dump(nb,file)